In [10]:
import warnings, numpy as np, pandas as pd, pymysql, matplotlib, matplotlib.pyplot as plt, seaborn as sb, geopandas as gp, geopy
from datetime import datetime, timedelta

def detect_parks_grid_in_time(database_filename,overall_list,date_hour_list,latitude_range,longitude_range,min_threshold,num_cell_v=20,num_cell_h=20): 
    
    grid_ticks_v = []
    grid_ticks_h = []
    
    grid_ticks_v.append(latitude_range[0])
    grid_ticks_h.append(longitude_range[0])
    
    delta_v = (latitude_range[1] - latitude_range[0])/float(num_cell_v)
    delta_h = (longitude_range[1] - longitude_range[0])/float(num_cell_h)
    
    for i1 in range(0,num_cell_h):
        grid_ticks_h.append(grid_ticks_h[i1]+delta_h)
    
    for i1 in range(0,num_cell_v):
        grid_ticks_v.append(grid_ticks_v[i1]+delta_v)
    
    lonbins = None
    latbins = None
    
    with warnings.catch_warnings():
        warnings.filterwarnings('error')
        for date_hour in date_hour_list:

            in_hour = date_hour[0]
            fin_hour = date_hour[1]
            query_string = """SELECT car_id, ST_X(start_pos) AS start_lon, ST_Y(start_pos) AS start_lat, start_time, end_time, 
            nr_min FROM %s WHERE  start_time > "%s" and end_time < "%s" """ % (database_filename,in_hour,fin_hour)

            connection = pymysql.connect('194.116.76.192', 'bd7', 'bd7', 'final_project')

            df=pd.read_sql(query_string, connection, parse_dates=["start_time", "end_time"])

            good_records = df.loc[(df['start_lat'] > latitude_range[0]) & (df['start_lat'] < latitude_range[1]) & 
                                 (df['start_lon'] > longitude_range[0]) & (df['start_lon'] < longitude_range[1]) & 
                                 (df['nr_min'] > min_threshold)].copy()

            (good_records['long_bin'], lonbins) = pd.cut(good_records['start_lon'], grid_ticks_h, retbins=True)
            good_records['long_bin'] = pd.to_numeric(good_records['long_bin'].apply(lambda x: (x.left+x.right)/2.0))
            (good_records['lat_bin'], latbins) = pd.cut(good_records['start_lat'], grid_ticks_v, retbins=True)
            good_records['lat_bin'] = pd.to_numeric(good_records['lat_bin'].apply(lambda x: (x.left+x.right)/2.0))

            time_list = []
            time_list.append(datetime.strptime(in_hour, "%Y-%m-%d %H:%M"))

            for i1 in range(1,25):
                time_list.append(time_list[0] + timedelta(hours=i1))

            for i1 in range(0,24):
                records_per_hour = good_records.loc[(good_records['start_time'] > time_list[i1]) & (good_records['end_time'] < time_list[i1+1])]

                good_records_formatted = records_per_hour[['long_bin','lat_bin']]

                grouped_good_records = good_records_formatted.groupby(['long_bin','lat_bin'])

                for key, item in grouped_good_records:
                    overall_list.append([key[0],key[1],grouped_good_records.get_group(key).shape[0],int(time_list[i1].strftime('%d')),time_list[i1].weekday(),time_list[i1].hour])


        return overall_list 



start_date_1 = '2017-06-01 00:00'
start_date_2 = '2017-06-02 00:00'
date_list_2 = []
date_list_2.append([start_date_1,start_date_2])

for i1 in range(1,30):
    date_list_2.append([(datetime.strptime(date_list_2[0][0], "%Y-%m-%d %H:%M") + timedelta(days=i1)).strftime("%Y-%m-%d %H:%M"),(datetime.strptime(date_list_2[0][1], "%Y-%m-%d %H:%M") + timedelta(days=i1)).strftime("%Y-%m-%d %H:%M")])

results = [['Long','Lat','N. of parkings','Day of the month','Day of the week','Hour']]    
    
results = detect_parks_grid_in_time('trips3',results,date_list_2,[44.9941845,45.1202965],[7.5991039,7.7697372],20)	

res = pd.DataFrame(results)
    
res.to_csv('/mnt/docs/Progetti/BigDive/Workspace/Final_Project/grid_result.csv',index=False,header=False)